## Canny Edge Detector

1.Filter image with x, y derivatives of Gaussian	

2. Find	magnitude and orientation of gradient	

3. Non maximum suppression   
* Consider the gradient's direction. Should be the direction of biggest change. if it's local maxima, then declare it as "edge". otherwise gradient= 0
<img src='Non-max-suppression.jpg'>

4. Thresholding	and	linking	(hysteresis):		
* If the gradient at a pixel is:
    - above "high", declare it as an "edge pixel"
    - below "low", declate it as a "non-edge"
    - between "low" and "high"
         if any neighbor is edge, then declare it as an edge; otherwise "non-edge"


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

∇ Gradient
Δ Laplacian

In [ ]:
mu = 10
sigma = 5
noise = np.random.normal(mu, sigma, 100) 
A = [item for item in [0,255] for i in range(50)]

In [ ]:
B = A + noise
print(B)

In [ ]:
plt.plot(B)

In [ ]:
plt.plot(B[1:]/B[0:(len(B)-1)])

In [ ]:
from scipy.ndimage.filters import gaussian_filter

In [ ]:
C = gaussian_filter(B, sigma, order=0, output=None, mode='reflect', cval=0.0, truncate=4.0)

In [ ]:
plt.plot(C)

In [ ]:
plt.plot(C[1:]/C[0:(len(C)-1)])

## A model fitting method for edge detection	
– RANSAC (RANdom SAmple Consensus): is an iterative method for estimating a mathematical model from a data set that contains outliers.  

RANSAC is accomplished with the following steps:  
    1. Randomly selecting a subset of the data set
    2. Fitting a model to the selected subset
    3. Determining the number of outliers
    4. Repeating steps 1-3 for a prescribed number of iterations


In [ ]:


fig, ax = plt.subplots()
# fig.set_tight_layout(True)

# Query the figure's on-screen size and DPI. Note that when saving the figure to
# a file, we need to provide a DPI for that separately.
print('fig size: {0} DPI, size in inches {1}'.format(
    fig.get_dpi(), fig.get_size_inches()))
def init():
    line.set_data([], [])
    return (line,)

# Plot a scatter that persists (isn't redrawn) and the initial line.
x = np.arange(0, 20, 0.1)
ax.scatter(x, x + np.random.normal(0, 3.0, len(x)))
line, = ax.plot(x, x - 5, 'r-', linewidth=2)


def update(i):
    label = 'timestep {0}'.format(i)
    print(label)
    # Update the line and the axes (with a new xlabel). Return a tuple of
    # "artists" that have to be redrawn for this frame.
    line.set_ydata(x - 5 + i)
    ax.set_xlabel(label)
    return line, ax

anim = animation.FuncAnimation(fig, update, frames=np.arange(0, 10), interval=200)
HTML(anim.to_html5_video())


# if __name__ == '__main__':
    # FuncAnimation will call the 'update' function for each frame; here
    # animating over 10 frames, with an interval of 200ms between frames.
    
#     if len(sys.argv) > 1 and sys.argv[1] == 'save':
#         anim.save('line.gif', dpi=80, writer='imagemagick')
#     else:
        # plt.show() will just loop the animation forever.
#         plt.show()
        
# anim = animation.FuncAnimation(fig, animate, init_func=init,
#                                frames=N, interval=20, blit=True)
# anim

In [ ]:
import numpy as np
import scipy
import matplotlib.pyplot as plt
import math
import sys
 
# Ransac parameters
ransac_iterations = 20  # number of iterations
ransac_threshold = 3    # threshold
ransac_ratio = 0.6      # ratio of inliers required to assert
                        # that a model fits well to data
# generate sparse input data
n_samples = 500               # number of input points
outliers_ratio = 0.4          # ratio of outliers
 
n_inputs = 1
n_outputs = 1
 
# generate samples
x = 30*np.random.random((n_samples,n_inputs) )
 
# generate line's slope (called here perfect fit)
perfect_fit = 0.5*np.random.normal(size=(n_inputs,n_outputs) )
 
# compute output
y = scipy.dot(x,perfect_fit)

In [ ]:
# add a little gaussian noise
x_noise = x + np.random.normal(size=x.shape)
y_noise = y + np.random.normal(size=y.shape)
 
# add some outliers to the point-set
n_outliers = int(outliers_ratio*n_samples)
indices = np.arange(x_noise.shape[0])
np.random.shuffle(indices)
outlier_indices = indices[:n_outliers]
 
x_noise[outlier_indices] = 30*np.random.random(size=(n_outliers,n_inputs))
 
# gaussian outliers
y_noise[outlier_indices] = 30*np.random.normal(size=(n_outliers,n_outputs))

In [ ]:
def find_line_model(points):
    """ find a line model for the given points
    :param points selected points for model fitting
    :return line model
    """
 
    # [WARNING] vertical and horizontal lines should be treated differently
    #           here we just add some noise to avoid division by zero
 
    # find a line model for these points
    m = (points[1,1] - points[0,1]) / (points[1,0] - points[0,0] + sys.float_info.epsilon)  # slope (gradient) of the line
    c = points[1,1] - m * points[1,0]                                     # y-intercept of the line
 
    return m, c

def find_intercept_point(m, c, x0, y0):
    """ find an intercept point of the line model with
        a normal from point (x0,y0) to it
    :param m slope of the line model
    :param c y-intercept of the line model
    :param x0 point's x coordinate
    :param y0 point's y coordinate
    :return intercept point
    """
 
    # intersection point with the model
    x = (x0 + m*y0 - m*c)/(1 + m**2)
    y = (m*x0 + (m**2)*y0 - (m**2)*c)/(1 + m**2) + c
 
    return x, y

In [ ]:
def ransac_plot(n, x, y, m, c, final=False, x_in=(), y_in=(), points=()):
    """ plot the current RANSAC step
    :param n      iteration
    :param points picked up points for modeling
    :param x      samples x
    :param y      samples y
    :param m      slope of the line model
    :param c      shift of the line model
    :param x_in   inliers x
    :param y_in   inliers y
    """
 
    fname = "output/figure_" + str(n) + ".png"
    line_width = 1.
    line_color = '#0080ff'
    title = 'iteration ' + str(n)
 
    if final:
        fname = "output/final.png"
        line_width = 3.
        line_color = '#ff0000'
        title = 'final solution'
 
    plt.figure("Ransac", figsize=(15., 15.))
 
    # grid for the plot
    grid = [min(x) - 10, max(x) + 10, min(y) - 20, max(y) + 20]
    plt.axis(grid)
 
    # put grid on the plot
    plt.grid(b=True, which='major', color='0.75', linestyle='--')
    plt.xticks([i for i in range(int(min(x)) - 10, int(max(x)) + 10, 5)])
    plt.yticks([i for i in range(int(min(y)) - 20, int(max(y)) + 20, 10)])
 
    # plot input points
    plt.plot(x[:,0], y[:,0], marker='o', label='Input points', color='#00cc00', linestyle='None', alpha=0.4)
 
    # draw the current model
    plt.plot(x, m*x + c, 'r', label='Line model', color=line_color, linewidth=line_width)
 
    # draw inliers
    if not final:
        plt.plot(x_in, y_in, marker='o', label='Inliers', linestyle='None', color='#ff0000', alpha=0.6)
 
    # draw points picked up for the modeling
    if not final:
        plt.plot(points[:,0], points[:,1], marker='o', label='Picked points', color='#0000cc', linestyle='None', alpha=0.6)
 
    plt.title(title)
    plt.legend()
    plt.savefig(fname)
    plt.close()

In [ ]:
data = np.hstack( (x_noise,y_noise) )
 
ratio = 0.
model_m = 0.
model_c = 0.
 
# perform RANSAC iterations
for it in range(ransac_iterations):
 
    # pick up two random points
    n = 2
 
    all_indices = np.arange(x_noise.shape[0])
    np.random.shuffle(all_indices)
 
    indices_1 = all_indices[:n]
    indices_2 = all_indices[n:]
 
    maybe_points = data[indices_1,:]
    test_points = data[indices_2,:]
 
    # find a line model for these points
    m, c = find_line_model(maybe_points)
 
    x_list = []
    y_list = []
    num = 0
 
    # find orthogonal lines to the model for all testing points
    for ind in range(test_points.shape[0]):
 
        x0 = test_points[ind,0]
        y0 = test_points[ind,1]
 
        # find an intercept point of the model with a normal from point (x0,y0)
        x1, y1 = find_intercept_point(m, c, x0, y0)
 
        # distance from point to the model
        dist = math.sqrt((x1 - x0)**2 + (y1 - y0)**2)
 
        # check whether it's an inlier or not
        if dist < ransac_threshold:
            x_list.append(x0)
            y_list.append(y0)
            num += 1
 
    x_inliers = np.array(x_list)
    y_inliers = np.array(y_list)
 
    # in case a new model is better - cache it
    if num/float(n_samples) > ratio:
        ratio = num/float(n_samples)
        model_m = m
        model_c = c
 
    print('  inlier ratio = ', num/float(n_samples))
    print('  model_m = ', model_m)
    print('  model_c = ', model_c)
 
    # plot the current step
    ransac_plot(it, x_noise,y_noise, m, c, False, x_inliers, y_inliers, maybe_points)
 
    # we are done in case we have enough inliers
    if num > n_samples*ransac_ratio:
        print('The model is found !')
        break
 
# plot the final model
ransac_plot(0, x_noise,y_noise, model_m, model_c, True)
 
print('\nFinal model:\n')
print('  ratio = ', ratio)
print('  model_m = ', model_m)
print('  model_c = ', model_c)

In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
# from matplotlib.animation import FuncAnimation
import seaborn
from matplotlib import rc, animation
from IPython.display import HTML

rc('animation', html='html5')
# First set up the figure, the axis, and the plot element we want to animate
fig, ax = plt.subplots()

ax.set_xlim(( 0, 2))
ax.set_ylim((-2, 2))

# ax.set_xlabel(label)
# ax.legend(loc='upper center')
line, = ax.plot([], [], lw=2)

In [3]:
def init():
    line.set_data([], [])
    return (line,)
# animation function. This is called sequentially


lines = []
timetext = ax.text(0.5,50,'')
plotlays, plotcols = [2,5], ["black","red"]
for index,lay in enumerate(plotlays):
    lobj = ax.plot([],[],lw=2,color=plotcols[index])[0]
    lines.append(lobj)
    
def animate(i):
    timetext.set_text(i)
    label = 'iteration%d'%i
    x = np.linspace(0, 2, 1000)
    y1 = np.sin(2 * np.pi * (x - 0.01 * i))
    y2= y1 +1
#     for lnum,line in enumerate(lines):
#         line.set_data(x,y1+i)
    line1.set_data(x, y1)
    line2.set_data(x, y2)
#     ax.plot(x,y1, 'k:', label='Data length')
#     ax.plot(x,y2, 'k--', label='Model length')
        # plot input points
    
#     ax.plot(x[:,0], y[:,0], marker='o', label='Input points', color='#00cc00', linestyle='None', alpha=0.4)
    # draw the current model
#     ax.plot(x, m*x + c, 'r', label='Line model', color=line_color, linewidth=line_width)
#     ax.plot(x_in, y_in, marker='o', label='Inliers', linestyle='None', color='#ff0000', alpha=0.6)
    
#     ax.set_xlabel(label)
#     ax.legend(loc='upper center')
#     ax.clear()
    return ([line1,line2],(timetext))
# call the animator. blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=20, interval=200, blit=True)

HTML(anim.to_html5_video())


NameError: name 'line1' is not defined